# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить два задания

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 5 баллов

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 5 баллов

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам.


__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [ ]:
import os
import abc
import time
import typing
import random
import warnings

import tqdm
import torch
import wandb
import numpy
import pandas
import torchscan
import torchvision
import sklearn.metrics
import matplotlib.pyplot as plt
import torch.utils.data as torchdata
from torchvision.transforms import v2 as transforms

device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

RANDOM_STATE = 42
def set_random_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
def fix_random():
    return set_random_seed(RANDOM_STATE)
fix_random()

In [ ]:
wandb.login(anonymous = "allow")

**✨ Внимание ✨**

В этом домашнем задании предлагается использовать библиотеку `pytorch_lightning`. Доступ к ее [документации](https://lightning.ai/docs/pytorch/stable/) заблокирован с территории РФ. Вы можете:

1. Получить к ней доступ с помощью VPN.

2. Собрать документацию самостоятельно. Для этого склонируйте [github-репозиторий](https://github.com/Lightning-AI/lightning/tree/master), запустите в нем терминал (на windows – git bash) и выполните команды:

```shell
git submodule update --init --recursive
make docs
```
После этого откройте появившийся файл `docs/build/html/index.html`. Для работы команд в вашем окружении должен быть `pip`. Полная инструкция [по ссылке](https://github.com/Lightning-AI/lightning/tree/master/docs).

3. Гуглить `<error message> pytorch lightning` или `<how to do this> pytorch lightning`. Stack overflow на территории РФ все еще доступен 😉

4. Не пользоваться `pytorch_lightning` и написать цикл обучения модели самостоятельно. Например, по аналогии с функцией `fit` из [семинара 4](https://github.com/hse-ds/iad-deep-learning/blob/master/2023/seminars/04.%20Optim%20%26%20Lightning/04_Optim%26Lightning_solution.ipynb).

## Задание 0

### Что поможет сделать на 10 из 10 (одно задание - 5 баллов)

1. Использовать все возможные методы оптимизации и эксперемнтировать с ними.
2. Подбор learning rate. Пример из прошлого семинара как это делать: [Как найти lr](https://pytorch-lightning.readthedocs.io/en/1.4.5/advanced/lr_finder.html)

```
  trainer = pl.Trainer(accelerator="gpu", max_epochs=2, auto_lr_find=True) 

  trainer.tune(module, train_dataloader, eval_dataloader)

  trainer.fit(module, train_dataloader, eval_dataloader))
```



3. Аугментация данных. [Документация (полезная)](https://pytorch.org/vision/main/transforms.html), а также [библиотека albumentation](https://towardsdatascience.com/getting-started-with-albumentation-winning-deep-learning-image-augmentation-technique-in-pytorch-47aaba0ee3f8)
4. Подбор архитектуры модели. 
5. Можно написать модель руками свою в YourNet, а можно импортировать не предобученную сетку известной архитектуры из модуля torchvision.models. Один из способов как можно сделать: 

  * `torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)`
  * Документация по возможным моделям и как их можно брать: [Документация (полезная)](https://pytorch.org/vision/stable/models.html)
6. Правильно нормализовывать данные при создании, пример [тык, но тут и в целом гайд от и до](https://www.pluralsight.com/guides/image-classification-with-pytorch)
7. Model Checkpointing. Сохраняйте свой прогресс (модели), чтобы когда что-то пойдет не так вы сможете начать с этого места или просто воспроизвести свои результаты модели, которые обучали. 
 * Пример как можно с wandb тут: [Сохраняем лучшие модели в wandb](https://docs.wandb.ai/guides/integrations/lightning)
 * По простому можно так: [Сохраняем модели в pytorch дока](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

### Подготовка данных

In [ ]:
class MysteriousDataset(torchdata.Dataset):
    def __init__(
            self,
            train: bool,
            preload: bool = True,
            precalculate_transform: bool = True,
            transform: typing.Optional[transforms.Compose] = None
        ):
        self.dataset_src = "./dataset/{}".format("train" if train else "val")
        self.dataset = torchvision.datasets.ImageFolder(self.dataset_src)
        self.precalculated_transform = None
        self.transform = None

        if preload or precalculate_transform:
            if precalculate_transform:
                self.precalculated_transform = transform
                self.transform = transform
                transform = None
            self.images, self.targets = self.load_all(True)
        self.transform = transform

    def load_all(self, progress_bar: bool = False):
        images = [ ]
        targets = [ ]
        for record in (tqdm.tqdm(self) if progress_bar else self):
            images.append(record[0])
            targets.append(record[1])
        try: return torch.stack(images), targets
        except: return images, targets

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if hasattr(self, 'images') and hasattr(self, 'targets'):
            image, target = self.images[idx], self.targets[idx]
        else:
            image, target = self.dataset[idx]
        if self.transform is not None:
            image = self.transform(image)
        return image, target
    
    def channel_stats(self):
        images, _ = self.load_all()
        return torch.mean(images, dim = [0, 2, 3]), torch.std(images, dim = [0, 2, 3])

In [ ]:
transform = transforms.Compose([ transforms.ToImage(), transforms.ToDtype(torch.float32, scale = True) ])

no_preload = MysteriousDataset(True, transform = transform, preload = False, precalculate_transform = False)
no_precalc = MysteriousDataset(True, transform = transform, preload = True, precalculate_transform = False)
preload = MysteriousDataset(True, transform = transform, preload = True, precalculate_transform = True)
torch_dataset = torchvision.datasets.ImageFolder("./dataset/train/", transform = transform)

In [ ]:
# Check that it is actually faster to loop through the dataset when it is fully in RAM
def test_loading(*args):
    for dataset, desc in args:
        for item in tqdm.tqdm(torchdata.DataLoader(dataset, batch_size = 256), desc = desc):
            pass

test_loading(
    (torch_dataset, 'torch_dataset'),
    (no_preload, 'no_preload'),
    (no_precalc, 'no_precalc'),
    (preload, 'preload')
)

#### Посчитаем поканальные средние и стандартные отклонения для нормализации данных

In [ ]:
mean, std = preload.channel_stats()
print(mean, std)

In [ ]:
del no_preload
del no_precalc
del preload
del torch_dataset

#### Создадим датасеты

In [ ]:
fix_random()

# Transforms
transform = transforms.Compose([
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale = True),
    transforms.Normalize(mean, std)
])

# Load datasets
train_set = MysteriousDataset(train = True, preload = True, precalculate_transform = True, transform = transform)
test_set = MysteriousDataset(train = False, preload = True, precalculate_transform = True, transform = transform)

# Check
print(len(train_set), len(test_set))

# Just very simple sanity checks
assert isinstance(train_set[0], tuple)
assert len(train_set[0]) == 2
assert isinstance(train_set[1][1], int)
assert isinstance(train_set[1][0], torch.Tensor)
assert train_set[1][0].shape == torch.Size([ 3, 64, 64 ])
print("Dataset tests passed")

for images, targets in torchdata.DataLoader(train_set, batch_size = 256):
    assert isinstance(images, torch.Tensor)
    assert isinstance(targets, torch.Tensor)
    assert images.shape == torch.Size([ 256, 3, 64, 64 ])
    assert targets.shape == torch.Size([ 256 ])
    print("DataLoader tests passed")
    break

### Посмотрим на картиночки

In [ ]:
fix_random()

# Denormalization
denormalize = transforms.Compose([
    transforms.Normalize(mean = [ 0., 0., 0. ], std = 1 / std),
    transforms.Normalize(mean = -mean, std = [ 1., 1., 1. ])
])

# Display some samples from each dataset
def display_examples(dataset, row):
    train_loader = torchdata.DataLoader(dataset, batch_size = 10, shuffle = True)
    for i, (image, label) in enumerate(zip(*next(iter(train_loader)))):
        plt.subplot(3, 10, i + 10 * (row - 1) + 1)
        plt.axis('off')
        plt.title('{}'.format(label))
        plt.imshow((denormalize(image).permute(1, 2, 0).numpy() * 255).astype(numpy.uint8))

plt.rcParams["figure.figsize"] = (15, 5)
display_examples(train_set, 1)
display_examples(test_set, 2)

## Задание 1. 

5 баллов
Добейтесь accuracy на валидации не менее 0.44. В этом задании запрещено пользоваться предобученными моделями и ресайзом картинок.


Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались)
4. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

In [ ]:
class BaseClassifier(abc.ABC):
    @abc.abstractmethod
    def fit(self, dataset: torchdata.Dataset):
        raise NotImplementedError

    @abc.abstractmethod
    def predict(self, images: torch.Tensor) -> typing.Tuple[torch.Tensor, torch.Tensor]:
        raise NotImplementedError
    
    def calc_metrics(self, dataset: torchdata.Dataset) -> dict:
        all_true = torch.tensor([])
        all_scores = torch.empty((0, 200))
        all_predictions = torch.tensor([])
        loader = torchdata.DataLoader(dataset, batch_size = 256, shuffle = False)
        for images, labels in loader:
            all_true = torch.cat([ all_true, labels ])
            predictions, scores = self.predict(images)
            all_scores = torch.cat([ all_scores, scores.detach().cpu() ])
            all_predictions = torch.cat([ all_predictions, predictions.detach().cpu() ])

        return {
            'Accuracy':       sklearn.metrics.accuracy_score      (all_true, all_predictions),
            'TOP-2 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 2),
            'TOP-3 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 3),
            'TOP-4 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 4),
            'TOP-5 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 5),
            # 'TOP-6 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 6),
            # 'TOP-7 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 7),
            # 'TOP-8 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 8),
            # 'TOP-9 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 9),
            # 'AUC-ROC':        sklearn.metrics.roc_auc_score       (all_true, all_scores, multi_class = 'ovo'),
            # 'Precision':      sklearn.metrics.precision_score     (all_true, all_predictions, average = 'macro'),
            # 'Recall':         sklearn.metrics.recall_score        (all_true, all_predictions, average = 'macro'),
            # 'F1-score':       sklearn.metrics.f1_score            (all_true, all_predictions, average = 'macro')
        }

In [ ]:
class Classifier(BaseClassifier):
    results = [ ]

    def __init__(
            self,
            name: str,
            model: torch.nn.Module,
            batch_size: int = 256,
            device: torch.device = device,
            optimizer: typing.Optional[torch.optim.Optimizer] = None,
            scheduler: typing.Optional[torch.optim.lr_scheduler.LRScheduler] = None,
        ):
        self.name = name
        self.history = [ ]
        self.device = device
        self.input_shape = None
        self.scheduler = scheduler
        self.batch_size = batch_size
        self.model = model.to(self.device)
        self.optimizer = optimizer or torch.optim.AdamW(self.model.parameters())


    def train(self, images: torch.Tensor, labels: torch.Tensor) -> float:
        self.model.train() # Enter train mode
        self.optimizer.zero_grad() # Zero gradients
        output = self.model(images.to(self.device)) # Get predictions
        loss = torch.nn.functional.cross_entropy(output, labels.to(self.device)) # Calculate loss
        loss.backward() # Calculate gradients
        self.optimizer.step() # Update weights
        return loss.item()

    def train_epoch(self, loader: torchdata.DataLoader) -> float:
        sum_loss = 0
        for images, labels in loader:
            sum_loss += self.train(images, labels) # Train on one batch
        if self.scheduler is not None:
            self.scheduler.step()
        return sum_loss / len(loader) # Return average loss to avoid random-dependent graph
       
    def fit(self, train_set: torchdata.Dataset, val_set: torchdata.Dataset, n_epochs: int = 25):
        if self.input_shape is None:
            self.predict(train_set[0][0].unsqueeze(0)) # Initialize lazy layers and input shape
        loader = torchdata.DataLoader(train_set, batch_size = self.batch_size, shuffle = True)
        wandb.init(project = "DL-HW-2", name = self.name, anonymous = "allow")
        wandb.watch(self.model, log = "all")
        for epoch in tqdm.trange(n_epochs):
            # Train
            train_start = time.perf_counter()
            loss = self.train_epoch(loader)
            train_time = time.perf_counter() - train_start

            # Validate
            val_start = time.perf_counter()
            metrics = self.calc_metrics(val_set)
            val_time = time.perf_counter() - val_start
            
            # Upload metrics
            metrics['Validation time'] = val_time
            metrics['Train time'] = train_time
            metrics['Loss'] = loss
            wandb.log(metrics)
            metrics['Epoch'] = epoch + 1
            self.history.append(metrics)

        # Finish the run
        wandb.finish()

        # Store best metrics
        best_metrics = max(self.history, key = lambda item: item['Accuracy'])
        Classifier.results.append({ **best_metrics, 'Name': self.name })

        return self
    

    def predict(self, images: torch.Tensor) -> typing.Tuple[torch.Tensor, torch.Tensor]:
        if self.input_shape is None:
            self.input_shape = images[0].shape # Lazily initialize input shape
        self.model.eval() # Enter evaluation mode
        with torch.no_grad():
            outputs = self.model(images.to(self.device)) # Get outputs
            scores = torch.softmax(outputs, dim = 1) # Make probabilities
            predictions = torch.argmax(scores, dim = 1) # Calculate predictions
        return predictions, scores
    

    def summary(self):
        warnings.filterwarnings("ignore")
        display(pandas.DataFrame(Classifier.results).set_index("Name"))
        torchscan.summary(self.model.eval(), self.input_shape, receptive_field = True)

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(16), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(32), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(64), torch.nn.GELU(),
    torch.nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(128), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(256), torch.nn.GELU(),
    torch.nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1), torch.nn.BatchNorm2d(512), torch.nn.GELU(), torch.nn.MaxPool2d(2, 2),

    torch.nn.Flatten(), torch.nn.Dropout(0.5), torch.nn.LazyLinear(1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(),
    torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200)
)
final_model = Classifier('First model', model).fit(train_set, test_set, 25)
final_model.summary()

## Task 2

In [ ]:
class BaseClassifier(abc.ABC):
    @abc.abstractmethod
    def fit(self, dataset: torchdata.Dataset):
        raise NotImplementedError

    @abc.abstractmethod
    def predict(self, images: torch.Tensor) -> typing.Tuple[torch.Tensor, torch.Tensor]:
        raise NotImplementedError
    
    def calc_metrics(self, dataset: torchdata.Dataset) -> dict:
        all_true = torch.tensor([])
        all_scores = torch.empty((0, 200))
        all_predictions = torch.tensor([])
        loader = torchdata.DataLoader(dataset, batch_size = 256, shuffle = False)
        for images, labels in loader:
            all_true = torch.cat([ all_true, labels ])
            predictions, scores = self.predict(images)
            all_scores = torch.cat([ all_scores, scores.detach().cpu() ])
            all_predictions = torch.cat([ all_predictions, predictions.detach().cpu() ])

        return {
            'Accuracy':       sklearn.metrics.accuracy_score      (all_true, all_predictions),
            'TOP-2 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 2),
            'TOP-3 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 3),
            'TOP-4 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 4),
            'TOP-5 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 5),
            # 'TOP-6 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 6),
            # 'TOP-7 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 7),
            # 'TOP-8 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 8),
            # 'TOP-9 Accuracy': sklearn.metrics.top_k_accuracy_score(all_true, all_scores, k = 9),
            # 'AUC-ROC':        sklearn.metrics.roc_auc_score       (all_true, all_scores, multi_class = 'ovo'),
            # 'Precision':      sklearn.metrics.precision_score     (all_true, all_predictions, average = 'macro'),
            # 'Recall':         sklearn.metrics.recall_score        (all_true, all_predictions, average = 'macro'),
            # 'F1-score':       sklearn.metrics.f1_score            (all_true, all_predictions, average = 'macro')
        }

class Classifier(BaseClassifier):
    results = [ ]

    def __init__(
            self,
            name: str,
            model: torch.nn.Module,
            batch_size: int = 256,
            device: torch.device = device,
            optimizer: typing.Optional[torch.optim.Optimizer] = None,
            scheduler: typing.Optional[torch.optim.lr_scheduler.LRScheduler] = None,
        ):
        self.name = name
        self.history = [ ]
        self.device = device
        self.input_shape = None
        self.scheduler = scheduler
        self.batch_size = batch_size
        self.model = model.to(self.device)
        self.optimizer = optimizer or torch.optim.AdamW(self.model.parameters())


    def train(self, images: torch.Tensor, labels: torch.Tensor) -> float:
        self.model.train() # Enter train mode
        self.optimizer.zero_grad() # Zero gradients
        output = self.model(images.to(self.device)) # Get predictions
        loss = torch.nn.functional.cross_entropy(output, labels.to(self.device)) # Calculate loss
        loss.backward() # Calculate gradients
        self.optimizer.step() # Update weights
        return loss.item()

    def train_epoch(self, loader: torchdata.DataLoader) -> float:
        sum_loss = 0
        for images, labels in loader:
            sum_loss += self.train(images, labels) # Train on one batch
        if self.scheduler is not None:
            self.scheduler.step()
        return sum_loss / len(loader) # Return average loss to avoid random-dependent graph
       
    def fit(self, train_set: torchdata.Dataset, val_set: torchdata.Dataset, n_epochs: int = 25):
        if self.input_shape is None:
            self.predict(train_set[0][0].unsqueeze(0)) # Initialize lazy layers and input shape
        loader = torchdata.DataLoader(train_set, batch_size = self.batch_size, shuffle = True)
        wandb.init(project = "DL-HW-2", name = self.name, anonymous = "allow")
        wandb.watch(self.model, log = "all")
        for epoch in tqdm.trange(n_epochs):
            # Train
            train_start = time.perf_counter()
            loss = self.train_epoch(loader)
            train_time = time.perf_counter() - train_start

            # Validate
            val_start = time.perf_counter()
            metrics = self.calc_metrics(val_set)
            val_time = time.perf_counter() - val_start
            
            # Upload metrics
            metrics['Validation time'] = val_time
            metrics['Train time'] = train_time
            metrics['Loss'] = loss
            wandb.log(metrics)
            metrics['Epoch'] = epoch + 1
            self.history.append(metrics)

        # Finish the run
        wandb.finish()

        # Store best metrics
        best_metrics = max(self.history, key = lambda item: item['Accuracy'])
        Classifier.results.append({ **best_metrics, 'Name': self.name })

        return self
    

    def predict(self, images: torch.Tensor) -> typing.Tuple[torch.Tensor, torch.Tensor]:
        if self.input_shape is None:
            self.input_shape = images[0].shape # Lazily initialize input shape
        self.model.eval() # Enter evaluation mode
        with torch.no_grad():
            outputs = self.model(images.to(self.device)) # Get outputs
            scores = torch.softmax(outputs, dim = 1) # Make probabilities
            predictions = torch.argmax(scores, dim = 1) # Calculate predictions
        return predictions, scores
    

    def summary(self):
        warnings.filterwarnings("ignore")
        display(pandas.DataFrame(Classifier.results).set_index("Name"))
        torchscan.summary(self.model.eval(), self.input_shape, receptive_field = True)

In [ ]:
# https://github.com/pytorch/vision/issues/7744
def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return torch.hub.load_state_dict_from_url(self.url, *args, **kwargs)
torchvision.models._api.WeightsEnum.get_state_dict = get_state_dict

In [ ]:
mean = torch.tensor([0.4802, 0.4481, 0.3975])
std = torch.tensor([0.2764, 0.2689, 0.2816])

In [ ]:
fix_random()
class MysteriousDatasetWithPretrained(torchdata.Dataset):
    def __init__(
            self,
            train: bool,
            transform: typing.Optional[transforms.Compose] = None
        ):
        dataset_src = "./dataset/{}".format("train" if train else "val")
        dataset = torchvision.datasets.ImageFolder(dataset_src, transform = transform)

        basemodel = torchvision.models.resnext101_64x4d(weights = torchvision.models.ResNeXt101_64X4D_Weights.IMAGENET1K_V1).to(device)
        basemodel.fc = torch.nn.Identity()
        basemodel.eval()

        with torch.no_grad():
            self.images = [ ]
            self.targets = [ ]
            for images, targets in tqdm.tqdm(torchdata.DataLoader(dataset, batch_size = 128)):
                self.images.append(basemodel(images.to(device)).to('cpu'))
                self.targets.append(targets)
            self.images = torch.cat(self.images)
            self.targets = torch.cat(self.targets)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.images[idx], self.targets[idx]
    
# Transforms
test_transform = transforms.Compose([
    transforms.Resize((232, 232)),
    transforms.CenterCrop((224, 224)),
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale = True),
    transforms.Normalize(mean, std)
])
test_transform = torchvision.models.ResNeXt101_64X4D_Weights.IMAGENET1K_V1.transforms()
train_transform = transforms.Compose([
    # transforms.RandomCrop(32, padding = 4),
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomRotation(15),
    test_transform
])

# Load datasets
# train_set = MysteriousDataset(train = True, preload = True, precalculate_transform = False, transform = train_transform)
# test_set = MysteriousDataset(train = False, preload = True, precalculate_transform = True, transform = test_transform)

train_set_pr = MysteriousDatasetWithPretrained(train = True, transform = train_transform)
test_set_pr = MysteriousDatasetWithPretrained(train = False, transform = test_transform)

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Linear(2048, 1024), torch.nn.GELU(), torch.nn.Linear(1024, 200)
)

optimizer = torch.optim.AdamW(model.parameters(), lr = 3e-5)
resnet_model = Classifier('resnext101_64x4d', model, optimizer = optimizer)
resnet_model.fit(train_set_pr, test_set_pr, 100)
resnet_model.calc_metrics(test_set_pr)

In [ ]:
wandb.finish()
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
class AugmentedDataset(torchdata.Dataset):
    def __init__(self, datasets: typing.List[MysteriousDatasetWithPretrained]):
        self.datasets = datasets

    def __len__(self):
        return len(self.datasets[0])

    def __getitem__(self, idx):
        return random.choice(self.datasets)[idx]

augm_train_set = AugmentedDataset([
    train_set_pr,
    *(
        MysteriousDatasetWithPretrained(
            train = True,
            transform = transforms.Compose([ transforms.RandAugment(), test_transform ])
        )
        for i in range(0, 3)
    )
])

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Linear(2048, 200)
)

optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4)
resnet_model = Classifier('ResNet152', model, optimizer = optimizer)
resnet_model.fit(augm_train_set, test_set_pr, 100)
resnet_model.calc_metrics(test_set_pr)

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.BatchNorm1d(2048), torch.nn.Linear(2048, 200)
)

optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.97)
resnet_model = Classifier('Resnet50', model, optimizer = optimizer, scheduler = scheduler)
resnet_model.fit(augm_train_set, test_set_pr, 100)
resnet_model.calc_metrics(test_set_pr)

In [ ]:
fix_random()
model = torch.nn.Sequential(
    torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), torch.nn.ReLU(), torch.nn.Linear(1024, 200)
)

optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4)
scheduler= torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.75)
resnet_model = Classifier('Resnet50', model, optimizer = optimizer, scheduler = scheduler)
resnet_model.fit(augm_train_set, test_set_pr, 100)
resnet_model.calc_metrics(test_set_pr)

In [ ]:
fix_random()
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V2)
        self.pretrained.fc = torch.nn.Identity()
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(0.5), torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), torch.nn.GELU(),
            torch.nn.Dropout(0.5), torch.nn.Linear(1024, 200)
        )
    def forward(self, *args, **kwargs):
        with torch.no_grad():
            features = self.pretrained(*args, **kwargs)
        return self.model(features)

resnet_model = Classifier('Resnet50', Net())
resnet_model.fit(train_set, test_set, 25)
resnet_model.calc_metrics(test_set)

In [ ]:
del resnet_model

In [ ]:
wandb.finish()
import gc
gc.collect()
torch.cuda.empty_cache()

### Модель (или просто импортируйте не предобученную)

In [ ]:
class YourNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        pass

    def _forward(self, x):
        # runs the Neural Network
        # YOUR CODE HERE
        pass

    def forward(self, images, target=None):
        # YOUR CODE HERE
        pass

    def get_accuracy(self, reset=False):
        # YOUR CODE HERE
        pass

### Тренировочный класс lightning

In [ ]:
class YourModule(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        # YOUR CODE HERE
        pass

    def forward(self, x):
        # YOUR CODE HERE
        pass

    def configure_optimizers(self):
        # YOUR CODE HERE
        pass

    def training_step(self, train_batch, batch_idx):
        # YOUR CODE HERE
        pass

    def validation_step(self, val_batch, batch_idx):
        # YOUR CODE HERE
        pass

In [ ]:
wandb_logger = WandbLogger(log_model='all') # какие возможности дает с pytorch_lightning https://docs.wandb.ai/guides/integrations/lightning
device = # YOUR CODE HERE

model = YourNet().to() # YOUR CODE HERE
module = YourModule() # YOUR CODE HERE

In [ ]:
trainer = pl.Trainer(logger=wandb_logger)  # YOUR CODE HERE
trainer.fit()  # YOUR CODE HERE

### Валидация результатов задания

In [ ]:
def evaluate_task(model, test_dataloader, device="cuda:0"):
    model = model.to(device)
    model.eval()
    accuracy = 0.0
    for images, labels in tqdm(test_dataloader):
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            loss = model(images, labels)
            acc_batch = model.get_accuracy(reset=True)
        accuracy += acc_batch
    accuracy = accuracy / len(test_dataloader)
    return accuracy

In [ ]:
model = # Подгрузить свою сохраненную модель сюда

accuracy = evaluate_task(model, val_dataloader)
print(f"Оценка за это задание составит {np.clip(10 * accuracy / 0.44, 0, 10):.2f} баллов")

## Задание 2

5 баллов
Добейтесь accuracy на валидации не менее 0.84. В этом задании делать ресайз и использовать претрейн можно.

Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались или можете пойти дальше).
4. Попробуйте сначала посмотреть качество исходной модели без дообучения, сохраните как baseline. Отсюда поймете какие слои нужно дообучать.
5. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

### Модель (или просто импортируйте предобученную)

In [ ]:
class YourNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        pass

    def _forward(self, x):
        # runs the Neural Network
        # YOUR CODE HERE
        pass

    def forward(self, images, target=None):
        # YOUR CODE HERE
        pass

    def get_accuracy(self, reset=False):
        # YOUR CODE HERE
        pass

### Тренировочный класс lightning

In [ ]:
class YourModule(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        # YOUR CODE HERE
        pass

    def forward(self, x):
        # YOUR CODE HERE
        pass

    def configure_optimizers(self):
        # YOUR CODE HERE
        pass

    def training_step(self, train_batch, batch_idx):
        # YOUR CODE HERE
        pass

    def validation_step(self, val_batch, batch_idx):
        # YOUR CODE HERE
        pass

In [ ]:
wandb_logger = WandbLogger(log_model='all') # какие возможности дает с pytorch_lightning https://docs.wandb.ai/guides/integrations/lightning
device =  # YOUR CODE HERE

model = YourNet().to() # YOUR CODE HERE
module = YourModule() # YOUR CODE HERE

In [ ]:
trainer = pl.Trainer(logger=wandb_logger)  # YOUR CODE HERE
trainer.fit()  # YOUR CODE HERE

### Валидация результатов задания

In [ ]:
model = # Подгрузить свою сохраненную модель сюда

accuracy = evaluate_task(model, val_dataloader)
print(f"Оценка за это задание составит {np.clip(10 * (accuracy - 0.5) / 0.34, 0, 10):.2f} баллов")

# Отчёт об экспериментах 

текст писать тут (или ссылочку на wandb/любой трекер экспреиментов) для каждого задания, то есть не обязательно именно тут рисовать графики, если вы используете готовые трекеры/мониторинги ваших моделей.